<a href="https://colab.research.google.com/github/hyj12345/hyj12345/blob/main/BIS638_HW4_Ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Homework 4** 

## This homework assignment demonstrates how to use the API of EBI OLS to access/query the Disease Ontology to retrieve synonyms

##The code below searches all text fields that contain 'ICD9CM' and return iri, label and obo_id. The number of entries returned is determined by rowmax

In [ ]:
import requests 

#search for DOID terms that have ICD9CM codes  
# api-endpoint 
URL = "https://www.ebi.ac.uk/ols/api/search"  
# param values here 
searchterm='ICD9CM'
ont = "doid"
rowmax=50 #limit to 20 terms 
fieldlist='iri,label,obo_id'
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'q':searchterm, 'ontology':ont, 'rows':rowmax, 'fieldList':fieldlist} 
  
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
  
# extracting data in json format 
data = r.json() 
irilist=[]
for d in data['response']['docs']:
  print (d['obo_id'])
  print (d['iri'])
  irilist.append(d['iri'])
  print ('label: ', d['label'])


DOID:13575
http://purl.obolibrary.org/obo/DOID_13575
label:  non-renal secondary hyperparathyroidism
DOID:9754
http://purl.obolibrary.org/obo/DOID_9754
label:  ganglion or cyst of synovium/tendon/bursa
DOID:11283
http://purl.obolibrary.org/obo/DOID_11283
label:  peripheral scars of retina
DOID:11129
http://purl.obolibrary.org/obo/DOID_11129
label:  dislocation of ear ossicle
DOID:8646
http://purl.obolibrary.org/obo/DOID_8646
label:  substance-induced psychosis
DOID:13629
http://purl.obolibrary.org/obo/DOID_13629
label:  dentine erosion
DOID:9868
http://purl.obolibrary.org/obo/DOID_9868
label:  intestinal disaccharidase deficiency
DOID:13658
http://purl.obolibrary.org/obo/DOID_13658
label:  infective urethral stricture
DOID:561
http://purl.obolibrary.org/obo/DOID_561
label:  trigeminal nerve disease
DOID:251
http://purl.obolibrary.org/obo/DOID_251
label:  alcohol-induced mental disorder
DOID:1829
http://purl.obolibrary.org/obo/DOID_1829
label:  urethral stricture
DOID:12475
http://purl.

##The code below takes the list of iri generated by the above code and get the term information corresponding to each iri. The retrieved information for each term includes synonyms (if any) and ICD9 code.

In [ ]:
import pandas as pd

#the code below retrieve icd9 code for each iri
icdlist=[]
parentlist=[]
irilist2=[]
for iri in irilist:
  iri2=iri.replace(':','%253A')
  iri2=iri2.replace('/', '%252F')
  URL2='https://www.ebi.ac.uk/ols/api/ontologies/'+ont+'/terms/'+iri2
  r2 = requests.get(url = URL2)
  data2=r2.json()
  print ('iri', iri)
  if 'has_exact_synonym' in data2['annotation'].keys():
    for s in (data2['annotation']['has_exact_synonym']):
      print ('synonym:', s)

  for d2 in data2['obo_xref']:
    if d2['database']=='ICD9CM':
      icd_code=d2['id'].replace('.','')
      icdlist.append(icd_code)
      break 
  
data = {'iri':irilist,'icd9_code':icdlist}
df = pd.DataFrame(data)
#print (df)
#df.to_csv('/content/gdrive/My Drive/BIS638/doid_icd9.csv', index=False)


iri http://purl.obolibrary.org/obo/DOID_13575
iri http://purl.obolibrary.org/obo/DOID_9754
iri http://purl.obolibrary.org/obo/DOID_11283
iri http://purl.obolibrary.org/obo/DOID_11129
iri http://purl.obolibrary.org/obo/DOID_8646
iri http://purl.obolibrary.org/obo/DOID_13629
iri http://purl.obolibrary.org/obo/DOID_9868
iri http://purl.obolibrary.org/obo/DOID_13658
iri http://purl.obolibrary.org/obo/DOID_561
iri http://purl.obolibrary.org/obo/DOID_251
iri http://purl.obolibrary.org/obo/DOID_1829
iri http://purl.obolibrary.org/obo/DOID_12475
iri http://purl.obolibrary.org/obo/DOID_11184
iri http://purl.obolibrary.org/obo/DOID_1203
iri http://purl.obolibrary.org/obo/DOID_14140
iri http://purl.obolibrary.org/obo/DOID_312
iri http://purl.obolibrary.org/obo/DOID_14252
iri http://purl.obolibrary.org/obo/DOID_14253
iri http://purl.obolibrary.org/obo/DOID_14559
iri http://purl.obolibrary.org/obo/DOID_13239
iri http://purl.obolibrary.org/obo/DOID_9459
iri http://purl.obolibrary.org/obo/DOID_9407
i

## Modify the synonym retrieval code above to increase rowmax to 500 and generate two csv files: i) DOID to ICD9 code mappings and ii) synonyms file containing the DOID column and Synonym column. 

In [ ]:
import requests
import pandas as pd
# search for DOID terms that have ICD9CM codes
# api-endpoint 
URL = "https://www.ebi.ac.uk/ols/api/search"
# param values here 
searchterm = 'ICD9CM'
ont = "doid"
rowmax = 50  # limit to 20 terms
fieldlist = 'iri,label,obo_id'

# defining a params dict for the parameters to be sent to the API 
PARAMS = {'q': searchterm, 'ontology': ont, 'rows': rowmax, 'fieldList': fieldlist}

# sending get request and saving the response as response object 
r = requests.get(url=URL, params=PARAMS)

# extracting data in json format 
data = r.json()
print(data)
##第一张表 返回doid，icd9，label
#ontology_name: (doid)'obo_id': 'DOID:13575'  'label': 'non-renal secondary hyperparathyroidism'}
irilist=[]
lablelist=[]
obidlist=[]
for d in data['response']['docs']:
  irilist.append(d['iri'])
  lablelist.append(d['label'])
  obidlist.append(d['obo_id'].replace('DOID:', ''))

data1 = {'iri': irilist,'DOID':obidlist,'label': lablelist}
df1 = pd.DataFrame(data1)
df1.to_csv('/content/Assigment5/DOID_ICD9.csv', index=False)


{'responseHeader': {'status': 0, 'QTime': 4, 'params': {'facet.field': ['ontology_name', 'ontology_prefix', 'type', 'subset', 'is_defining_ontology', 'is_obsolete'], 'hl': 'true', 'fl': 'iri,label,obo_id', 'start': '0', 'fq': ['ontology_name: (doid)', 'is_obsolete:false'], 'rows': '50', 'hl.simple.pre': '<b>', 'bq': 'type:ontology^10.0 is_defining_ontology:true^100 label_s:"icd9cm"^5 synonym_s:"icd9cm"^3 annotations_trimmed:"icd9cm"', 'q': 'ICD9CM', 'defType': 'edismax', 'hl.simple.post': '</b>', 'qf': 'label^5 synonym^3 description short_form^2 obo_id^2 annotations logical_description iri', 'hl.fl': ['label', 'synonym', 'definition'], 'facet': 'true', 'wt': 'json'}}, 'response': {'numFound': 2113, 'start': 0, 'docs': [{'iri': 'http://purl.obolibrary.org/obo/DOID_13575', 'obo_id': 'DOID:13575', 'label': 'non-renal secondary hyperparathyroidism'}, {'iri': 'http://purl.obolibrary.org/obo/DOID_9754', 'obo_id': 'DOID:9754', 'label': 'ganglion or cyst of synovium/tendon/bursa'}, {'iri': 'ht

In [ ]:
import pandas as pd

# the code below retrieve icd9 code for each iri
icdlist = []
parentlist = []
irilist2 = []
synonymlist=[]
doidlist=[]
for iri in irilist:
    iri2 = iri.replace(':', '%253A')
    iri2 = iri2.replace('/', '%252F')
    
    URL2 = 'https://www.ebi.ac.uk/ols/api/ontologies/' + ont + '/terms/' + iri2
    # print('URL2', URL2)
    r2 = requests.get(url=URL2)
    data2 = r2.json()
    print('iri', iri)
    #synonym
    try:
        synonymlist.append(data2['annotation']['has_exact_synonym'])
    except KeyError:
        synonymlist.append('[null]')
    #DOID
    doidlist.append(data2['obo_id'].replace('DOID:', ''))

    #icd9
    for d2 in data2['obo_xref']:
        if d2['database'] == 'ICD9CM':
            icd_code = d2['id'].replace('.', '')
            icdlist.append(icd_code)
            break

data2 = {'iri': irilist,'DOID':doidlist,'ICD9': icdlist,'Synonym':synonymlist}
df2 = pd.DataFrame(data2)

df2.to_csv('/content/Assigment5/DOID_Synonym.csv', index=False)

iri http://purl.obolibrary.org/obo/DOID_13575
iri http://purl.obolibrary.org/obo/DOID_9754
iri http://purl.obolibrary.org/obo/DOID_11283
iri http://purl.obolibrary.org/obo/DOID_11129
iri http://purl.obolibrary.org/obo/DOID_8646
iri http://purl.obolibrary.org/obo/DOID_13629
iri http://purl.obolibrary.org/obo/DOID_9868
iri http://purl.obolibrary.org/obo/DOID_13658
iri http://purl.obolibrary.org/obo/DOID_561
iri http://purl.obolibrary.org/obo/DOID_251
iri http://purl.obolibrary.org/obo/DOID_1829
iri http://purl.obolibrary.org/obo/DOID_12475
iri http://purl.obolibrary.org/obo/DOID_11184
iri http://purl.obolibrary.org/obo/DOID_1203
iri http://purl.obolibrary.org/obo/DOID_14140
iri http://purl.obolibrary.org/obo/DOID_312
iri http://purl.obolibrary.org/obo/DOID_14252
iri http://purl.obolibrary.org/obo/DOID_14253
iri http://purl.obolibrary.org/obo/DOID_14559
iri http://purl.obolibrary.org/obo/DOID_13239
iri http://purl.obolibrary.org/obo/DOID_9459
iri http://purl.obolibrary.org/obo/DOID_9407
i

## Add two tables to the mimic database created in HW1. Load the two csv files into these new tables. Write a join query to identify patients with a diagnosis using the synonym table.

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
from IPython import display
display.Image("/content/gdrive/My Drive/BIS638/2021/mimic_doid2.png")

In [ ]:
CREATE TABLE Patients (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    gender varchar(5)  NOT NULL,
    dob datetime  NOT NULL,
    dod datetime  NOT NULL,
    dod_hosp datetime  NULL,
    dod_ssn datetime  NULL,
    expire_flag varchar(5)  NOT NULL,
    PRIMARY KEY (subject_id)
  );

CREATE TABLE Admissions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    admittime datetime  NOT NULL,
    dischtime datetime  NOT NULL,
    deathtime datetime   NULL,
    admission_type varchar(50)  NOT NULL,
    admission_location varchar(50)  NOT NULL,
    discharge_location varchar(50)  NOT NULL,
    insurance varchar(255)  NOT NULL,
    language varchar(10)   NULL,
    religion varchar(50)   NULL,
    marital_status varchar(50)   NULL,
    ethnicity varchar(200)  NOT NULL,
    edregtime datetime   NULL,
    edouttime datetime   NULL,
    diagnosis varchar(300)  NOT NULL,
    hospital_expire_flag int  NOT NULL,
    has_chartevents_data int  NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );

# insert code for creating D_icd_diagnoses 
CREATE TABLE D_icd_diagnoses (
    row_id int  NOT NULL,
    icd9_code varchar(10)  NOT NULL,
    short_title varchar(50)  NOT NULL,
    long_title varchar(300)  NOT NULL,
    PRIMARY KEY (icd9_code)
  );

# Insert code for creating Diagnoses_icd tables
CREATE TABLE Diagnoses_icd tables (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    ham_id int  NOT NULL,
    seq_num int  NOT NULL,
    icd9_code varchar(10)  NOT NULL
    PRIMARY KEY (row_id),
    FOREIGN KEY (icd9_code) REFERENCES D_icd_diagnoses(icd9_code),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)

  );

# Insert code for creating DOID_ICD9 tables 'DOID':obidlist,'label': lablelist
CREATE TABLE DOID_ICD9 tables (
    DOID varchar(50)  NOT NULL,
    label varchar(300)  NOT NULL
    PRIMARY KEY (DOID)
  );
# Insert code for creating Diagnoses_icd tables 'DOID':doidlist,'ICD9': icdlist,'Synonym':synonymlist
CREATE TABLE DOID_Synonym tables (
    DOID varchar(50)  NOT NULL,
    ICD9 varchar(50)  NOT NULL,
    Synonym varchar(300)
    PRIMARY KEY (DOID),
    FOREIGN KEY (DOID) REFERENCES DOID_ICD9(DOID),
    FOREIGN KEY (ICD9) REFERENCES D_icd_diagnoses(icd9_code)

  );

SyntaxError: ignored

In [46]:
import pandas as pd
import sqlite3
from sqlite3 import Error
from IPython.display import display


""" create a database connection to a database that resides
    in the drive
"""
path='/content/gdrive/My Drive/'
conn = None;
try:
  conn = sqlite3.connect(':memory:')
  #conn = sqlite3.connect(path+'mimic_demo.db')

  cursor = conn.cursor()
  #Doping tables if already exists.
  cursor.execute("DROP TABLE IF EXISTS Patients")
  cursor.execute("DROP TABLE IF EXISTS Admissions")
  cursor.execute("DROP TABLE IF EXISTS Diagnoses_icd")
  cursor.execute("DROP TABLE IF EXISTS D_icd_diagnoses")
  cursor.execute("DROP TABLE IF EXISTS DOID_ICD9")
  cursor.execute("DROP TABLE IF EXISTS DOID_Synonym")
  #Creating tables
  sql ='''CREATE TABLE Patients (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    gender varchar(5)  NOT NULL,
    dob datetime  NOT NULL,
    dod datetime  NOT NULL,
    dod_hosp datetime  NULL,
    dod_ssn datetime  NULL,
    expire_flag varchar(5)  NOT NULL,
    PRIMARY KEY (subject_id)
  );'''
  cursor.execute(sql)
  sql ='''CREATE TABLE Admissions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    admittime datetime  NOT NULL,
    dischtime datetime  NOT NULL,
    deathtime datetime   NULL,
    admission_type varchar(50)  NOT NULL,
    admission_location varchar(50)  NOT NULL,
    discharge_location varchar(50)  NOT NULL,
    insurance varchar(255)  NOT NULL,
    language varchar(10)   NULL,
    religion varchar(50)   NULL,
    marital_status varchar(50)   NULL,
    ethnicity varchar(200)  NOT NULL,
    edregtime datetime   NULL,
    edouttime datetime   NULL,
    diagnosis varchar(300)  NOT NULL,
    hospital_expire_flag int  NOT NULL,
    has_chartevents_data int  NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''
  cursor.execute(sql)
  #insert code to create D_icd_diagnoses table
  sql='''CREATE TABLE D_icd_diagnoses (
    row_id int  NOT NULL,
    icd9_code varchar(10)  NOT NULL,
    short_title varchar(50)  NOT NULL,
    long_title varchar(300)  NOT NULL,
    PRIMARY KEY (icd9_code)
  );'''
  cursor.execute(sql)
  #insert code to create Diagnoses_icd table
  sql='''
   CREATE TABLE Diagnoses_icd tables (
     row_id int  NOT NULL,
     subject_id int  NOT NULL,
     ham_id int  NOT NULL,
     seq_num int  NOT NULL,
     icd9_code varchar(10)  NOT NULL
     PRIMARY KEY (row_id),
     FOREIGN KEY (icd9_code) REFERENCES D_icd_diagnoses(subject_id),
     FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
   );'''

   # Insert code for creating DOID_ICD9 tables 'DOID':obidlist,'label': lablelist
  sql='''
   CREATE TABLE DOID_ICD9 tables (
    DOID varchar(50)  NOT NULL,
    label varchar(300)  NOT NULL
    PRIMARY KEY (DOID)
  );'''
# Insert code for creating Diagnoses_icd tables 'DOID':doidlist,'ICD9': icdlist,'Synonym':synonymlist
  sql='''
   CREATE TABLE DOID_Synonym tables (
    DOID varchar(50)  NOT NULL,
    ICD9 varchar(50)  NOT NULL,
    Synonym varchar(300)
    PRIMARY KEY (DOID),
    FOREIGN KEY (DOID) REFERENCES DOID_ICD9(DOID),
    FOREIGN KEY (ICD9) REFERENCES D_icd_diagnoses(icd9_code)
  );'''



  # cursor.execute(sql)


  print("Table(s) created successfully........")
  #conn.commit()

  #load the data from a csv file located in a directory (use your own directory into a Pandas DataFrame object
  patients = pd.read_csv(path+'PATIENTS.csv')
  admissions = pd.read_csv(path+'ADMISSIONS.csv')
  D_icd_diagnoses = pd.read_csv(path+'D_ICD_DIAGNOSES.csv')
  Diagnoses_icd = pd.read_csv(path+'DIAGNOSES_ICD.csv')
  doid_icd9 = pd.read_csv('/content/Assigment5/DOID_ICD9.csv').drop(['iri'],axis=1)
  doid_synonym = pd.read_csv('/content/Assigment5/DOID_Synonym.csv').drop(['iri'],axis=1)



  #write the data to a sqlite table
  patients.to_sql('Patients', conn, if_exists='replace', index = False)
  admissions.to_sql('Admissions', conn, if_exists='replace', index = False)
  D_icd_diagnoses.to_sql('D_icd_diagnoses', conn, if_exists='replace', index = False)
  doid_icd9.to_sql('DOID_ICD9', conn, if_exists='replace', index = False)
  doid_synonym.to_sql('DOID_Synonym', conn, if_exists='replace', index = False)

  #Commit your changes in the database
  conn.commit()
  #show hispanic or latino patients who were admitted to the hospital
  sql='''SELECT * 
  FROM D_icd_diagnoses D inner join doid_synonym d on D.icd9_code=d.ICD9
  '''
  patients = pd.read_sql(sql, conn)
  display (patients)
  




except Error as e:
    print(e)
finally:
    if conn:
        conn.close()


Table(s) created successfully........


,row_id,icd9_code,short_title,long_title,DOID,ICD9,Synonym
0,473,09841,Gonococcal iridocyclitis,Gonococcal iridocyclitis,9384,9841,[null]
1,480,09853,Gonococcal spondylitis,Gonococcal spondylitis,13127,9853,['Gonococcal spondylitis']
2,1966,1821,Mal neo uterine isthmus,Malignant neoplasm of isthmus,9459,1821,[null]
3,2228,2819,Deficiency anemia NOS,Unspecified deficiency anemia,13121,2819,['deficiency anemias']
4,2289,25202,Sec hyprprthyrd nonrenal,"Secondary hyperparathyroidism, non-renal",13575,25202,[null]
5,2672,3021,Zoophilia,Zoophilia,9336,3021,['Zoophilia']
6,2739,2713,Disaccharidase def/malab,Intestinal disaccharidase deficiencies and dis...,9868,2713,[null]
7,2817,3508,Trigeminal nerve dis NEC,Other specified trigeminal nerve disorders,561,3508,[null]
8,2843,34551,Part epil w intr epil,Localization-related (focal) (partial) epileps...,5129,34551,[null]
9,3070,32081,Anaerobic meningitis,Anaerobic meningitis,14559,32081,[null]
